In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install wandb -qqq
import wandb

In [ ]:
from sklearn.decomposition import PCA
import numpy as np
from keras.datasets import fashion_mnist
from sklearn.model_selection import train_test_split
from scipy.special import expit, logit
import pandas as pd

#One_Hot_Encoding():convert the given Data in one hot encoding in a 2D array 
def One_Hot_Encoding(Y, n_classes):
    src_arr = np.zeros((Y.shape[0],n_classes))
    for i in range(Y.shape[0]):
        Temp = [0.0]*n_classes
        Temp[Y[i]] = 1.0
        src_arr[i] = Temp
    return src_arr


#Get_Data(): To fetch Train/Test Data from the given file
def Get_Data(file, data, normalise = True, mean=0, std=1):
    print('Getting Data...')
    n_classes = 10
    
    df = pd.read_csv(file)
    
    df = df.iloc[np.random.permutation(len(df))]
  
    X = df[df.columns[0:-1]]
  
    
    mean = X.mean(axis=0)
    std  = X.std(axis = 0)
    
    if normalise == True:
      X = Normalise_Data(X, mean, std)
    
    X = X.values
  
    Y = df['label'].values
    
    
    print(data,' Data Fetched Successfully...')

    return (X, One_Hot_Encoding(Y, n_classes), mean, std)
#Normalise_Data():Normalizing the Given data
def Normalise_Data(data, mean, std):
    data = data - mean
    data = data/std
    return data




(X_train, Y_train, train_mean, train_std)=Get_Data('/content/drive/MyDrive/fashion/fashion-mnist_train.csv', data = 'Train', normalise = True )

X_Train, X_Valid, Y_Train, Y_Valid = train_test_split(X_train, Y_train, test_size=0.10, random_state=42)
(X_Test, Y_Test, train_mean, train_std)=Get_Data('/content/drive/MyDrive/fashion/fashion-mnist_test.csv', data = 'Test', normalise = True )


Getting Data...
Train  Data Fetched Successfully...
Getting Data...
Test  Data Fetched Successfully...


In [ ]:
import wandb
from re import X
import numpy as np
import warnings
from sklearn.metrics import log_loss
from scipy.special import expit, logit
from sklearn.metrics import mean_squared_error
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

#Class that initializes the initial parameters and contain the main operating functions  
class Feed_Forward_NeuralNetwork(object):
  number_of_neurons=[]
  number_of_layers=0
  Biases=[]
  Weights=[]
  Activation_gx=[]
  Layer_Output=[]
  Layer_Input=[]
  Loss_Function=""
  #Default fucntion to initialize parameters 
  def __init__(self,Layers,Activations,Loss_Funciton="CE",seed=8790,Weight_init='Random'):
    self.number_of_layers=len(Layers)
    self.number_of_neurons=Layers
    self.Activation_gx=Activations
    self.Loss_Function=Loss_Funciton
    if Weight_init=='Xavier':
      self.Weights=[np.random.randn(sizej,sizei)*np.sqrt(1/(sizei)) for sizei,sizej in zip(self.number_of_neurons[:-1],self.number_of_neurons[1:])]  
    else:
      self.Weights=[np.random.randn(sizej,sizei)*np.sqrt(2/(sizei+sizej)) for sizei,sizej in zip(self.number_of_neurons[:-1],self.number_of_neurons[1:])]
    self.Biases=[np.zeros((x,1)) for x in self.number_of_neurons[1:]]
    

  #Definition of alll the Activation Fucntion and there Derivatives
  def Sigmoid(self,x):
    return  expit(x)
  def stable_softmax(self,x):
    out = np.zeros(x.shape)
    for i in range(0, x.shape[1]):
      exps = np.exp(x[:, i] - np.max(x[:, i]))
      out[:, i] = exps / np.sum( exps)
    return out
  def Sigmoid_Drv(self, x):
    sigma_x = self.Sigmoid(x)
    return (sigma_x * (1 - sigma_x))
  def Relu(self,x):
    return np.maximum(0.0,x)
  def Tanh(self,x):
    return np.tanh(x)
  def Tanh_Drv(self, x):
    tanh_x = np.tanh(x)
    return (1 - np.square(tanh_x))
  def Relu_Drv(self, x):
    return np.greater(x,0).astype(int)

  #Feed_Network():Function that Implements Feed Forward Neural Network
  def Feed_Network(self,X_train):
    self.Layer_Input=[]
    self.Layer_Output=[]
    Weights=self.Weights
    biases=self.Biases
    no_of_datapoints=X_train.shape[0]
    no_of_features=X_train.shape[1]
    Temp=[]
    Activations=[]
    #Loop through all the layer and set the parameters
    for i in range(0,self.number_of_layers-1):

      if i==0:
        Temp.append(np.matmul(Weights[i],X_train.T)+biases[i])
      else:
        Temp.append(np.matmul(Weights[i],Activations[i-1])+biases[i])
      
      self.Layer_Input.append(Temp[i])

      #Check For Activation Fucnction
      if (self.Activation_gx[i]=='SoftMax') and (i == self.number_of_layers-2):
        Activations.append(self.stable_softmax(Temp[i]))
      elif self.Activation_gx[i]=='Tanh':
        Activations.append(self.Tanh(Temp[i]))
      elif self.Activation_gx[i]=='Sigmoid':
        Activations.append(self.Sigmoid(Temp[i]))
      elif self.Activation_gx[i]=='Relu':
        Activations.append(self.Relu(Temp[i]))
      



      self.Layer_Output.append(Activations[i])
    #Return the Probability distribution over the classes
    return self.Layer_Output[-1]
  #Backward_Pass():Implements Back Propogation Algorithm
  def Backward_Pass(self,X_train,Y_train):
    Gredient_wrt_weight=[0]*(len(self.Weights))
    Gradient_wrt_biases=[0]*(len(self.Biases))
    Gradient_wrt_layers=[0]*(self.number_of_layers-1)
    n=X_train.shape[0]


    for i in reversed(range(self.number_of_layers-1)):
      #Backdrop
      if i == self.number_of_layers-2:
        if self.Loss_Function=='CE':
          Gradient_wrt_layers[i]=self.Layer_Output[i]-Y_train.T
        elif self.Loss_Function=='MSE':
          Gradient_wrt_layers[i]=(self.Layer_Output[i]-Y_train.T)*self.Layer_Output[i]*(1-self.Layer_Output[i])

        Gredient_wrt_weight[i]=(1/n)*np.matmul(Gradient_wrt_layers[i],self.Layer_Output[i-1].T)
        Gradient_wrt_biases[i]=(1/n)*np.sum(Gradient_wrt_layers[i],axis=1,keepdims=True)
      #For the remaining layers other than the output layer
      else:
        activation_derivative=[]
        if self.Activation_gx[i]=='Tanh':
          activation_derivative=self.Tanh_Drv(Gradient_wrt_layers[i])
        elif self.Activation_gx[i]=='Sigmoid':
          activation_derivative=self.Sigmoid_Drv(Gradient_wrt_layers[i])
        elif self.Activation_gx[i]=='Relu':
          activation_derivative=self.Relu_Drv(Gradient_wrt_layers[i])
        
        Gradient_wrt_layers[i]=np.matmul(self.Weights[i+1].T,Gradient_wrt_layers[i+1])*activation_derivative#self.Tanh_Drv(Gradient_wrt_layers[i])
        
        if i==0:
          Gredient_wrt_weight[i]=(1/n)*np.matmul(Gradient_wrt_layers[i],X_train)
        else:
          Gredient_wrt_weight[i]=(1/n)*np.matmul(Gradient_wrt_layers[i],self.Layer_Output[i-1].T)
        
        Gradient_wrt_biases[i]=(1/n)*np.sum(Gradient_wrt_layers[i],axis=1,keepdims=True)
    # Return the gradient wrt to weights , biases ,layers
    return (Gradient_wrt_biases,Gredient_wrt_weight,Gradient_wrt_layers)
  #Compute the accuracy 
  def Get_Accuracy(self,y,y_pred):
    P=np.argmax(y,axis=0)
    Q=np.argmax(y_pred,axis=0)
    acc=np.sum(P == Q)/len(P)
    return 100*acc
  #Optimizer Fucntions definitions
  def GD(self,Grd_w,Grd_b,Eta):
    return (np.multiply(Eta,Grd_w),np.multiply(Eta,Grd_b))
  def Momentum(self,Momentum,Grd_w,Grd_b,Eta,prev_w,prev_b):
    upd_weight=np.multiply(Momentum,prev_w)+np.multiply(Eta,Grd_w)
    upd_biases=np.multiply(Momentum,prev_b)+np.multiply(Eta,Grd_b)
    return (upd_weight,upd_biases)
  def RMSprop(self,Grd_w,Grd_b,Eta):
    w_v,b_v=0,0
    eps=1e-3
    beta=0.9
    w_v=np.multiply(beta,w_v)
    b_v=np.multiply(beta,b_v)
    w_v=w_v+np.multiply(1-beta,np.power(Grd_w,2))
    b_v=b_v+np.multiply(1-beta,np.power(Grd_b,2))
    w_v_corrected=1/(np.power(w_v+eps,0.5))
    b_v_corrected=1/(np.power(b_v+eps,0.5))
    upd_w=np.multiply(w_v_corrected,Grd_w)*Eta
    upd_b=np.multiply(b_v_corrected,Grd_b)*Eta
    return (upd_w,upd_b)
  def Adam(self,Grd_w,Grd_b,Eta,t):
    w_m,b_m=0,0
    w_v,b_v=0,0
    beta1,beta2=0.9,0.99
    eps=1e-8
    w_v=np.multiply(beta2,w_v)
    b_v=np.multiply(beta2,b_v)
    w_v=w_v+np.multiply(1-beta2,np.power(Grd_w,2))
    b_v=b_v+np.multiply(1-beta2,np.power(Grd_b,2))
    w_v=w_v/(1-np.power(beta2,t+1))
    b_v=b_v/(1-np.power(beta2,t+1))
    w_m=np.multiply(beta1,w_m)
    b_m=np.multiply(beta1,b_m)
    w_m=w_m+np.multiply(1-beta1,Grd_w)
    b_m=b_m+np.multiply(1-beta1,Grd_b)
    w_m=w_m/(1-np.power(beta1,t+1))
    b_m=b_m/(1-np.power(beta1,t+1))
    upd_w=(Eta/np.power(w_v+eps,0.5))*w_m
    upd_b=(Eta/np.power(b_v+eps,0.5))*b_m

    return (upd_w,upd_b)
  def Nadam(self,Grd_w,Grd_b,Eta,t):
    w_m,b_m=0,0
    w_v,b_v=0,0
    beta1,beta2=0.9,0.99
    eps=1e-8
    w_v=np.multiply(beta2,w_v)
    b_v=np.multiply(beta2,b_v)
    w_v=w_v+np.multiply(1-beta2,np.power(Grd_w,2))
    b_v=b_v+np.multiply(1-beta2,np.power(Grd_b,2))
    w_m=np.multiply(beta1,w_m)
    b_m=np.multiply(beta1,b_m)
    w_m=w_m+np.multiply(1-beta1,Grd_w)
    b_m=b_m+np.multiply(1-beta1,Grd_b)
    w_m=np.multiply(beta1,w_m)/(1-beta1)+np.multiply(beta1,Grd_w)/(1-beta1)
    b_m=np.multiply(beta1,b_m)/(1-beta1)+np.multiply(beta1,Grd_b)/(1-beta1)
    w_v=np.multiply(np.multiply(beta2,w_v),1/(1-beta2))
    b_v=np.multiply(np.multiply(beta2,b_v),1/(1-beta2))
    upd_w=(Eta/np.power(w_v+eps,0.5))*w_m
    upd_b=(Eta/np.power(b_v+eps,0.5))*b_m 
                                
    return (upd_w,upd_b)
  def Nag(self,gamma,Grd_w,Grd_b,Eta,prev_w,prev_b):
    w_v,b_v=0,0
    w_v=np.multiply(gamma,prev_w)+np.multiply(Eta,Grd_w)
    b_v=np.multiply(gamma,prev_b)+np.multiply(Eta,Grd_b)
    prev_w=w_v 
    prev_b=b_v
    w_v=np.multiply(gamma,prev_w)
    b_v=np.multiply(gamma,prev_b)
    return w_v,b_v





  #Main_Training_Algo():It combines Feed forward neural Network and Back Propogation with optimizers
  def Main_Training_Algo(self,X_Valid,Y_Valid,X_Train,Y_Train,optimizer='Momentum',NEpochs=5,Batch_Size=16,Momentum=0.9,Eta=0.01,anneal=False):

    etap=Eta
    Loss_in_Validation=[]
    Loss_in_Training=[]
    update_weight=[]
    update_biases=[]
    prev_w=[np.zeros((sizej,sizei)) for sizei,sizej in zip(self.number_of_neurons[:-1],self.number_of_neurons[1:])]
    prev_b=[np.zeros((x,1)) for x in self.number_of_neurons[1:]]
    t=0
    for Ep in range(NEpochs):
      Loss_in_batch=0

      for start in range(0,X_Train.shape[0],Batch_Size):

        X_Train_curr_batch=X_Train[start:start+Batch_Size]
        Y_Train_curr_batch=Y_Train[start:start+Batch_Size]
        

        Curr_Batch_Prediction=self.Feed_Network(X_Train_curr_batch)

        (Gradient_wrt_biases,Gradient_wrt_weight,Gradient_wrt_layers)=self.Backward_Pass(X_Train_curr_batch,Y_Train_curr_batch)


        if optimizer=='GD':
          update_weight,update_biases=self.GD(Gradient_wrt_weight,Gradient_wrt_biases,Eta)
        elif optimizer=='Momentum':
          update_weight,update_biases=self.Momentum(Momentum,Gradient_wrt_weight,Gradient_wrt_biases,Eta,prev_w,prev_b)
          prev_w=update_weight
          prev_b=update_biases
        elif optimizer=='RMSprop':
          update_weight,update_biases=self.RMSprop(Gradient_wrt_weight,Gradient_wrt_biases,Eta)
        elif optimizer=='Adam':
          update_weight,update_biases=self.Adam(Gradient_wrt_weight,Gradient_wrt_biases,Eta,t)
        elif optimizer=='Nadam':
          update_weight,update_biases=self.Nadam(Gradient_wrt_weight,Gradient_wrt_biases,Eta,t)
        elif optimizer=='Nag':
          update_weight,update_biases=self.Nag(Momentum,Gradient_wrt_weight,Gradient_wrt_biases,Eta,prev_w,prev_b)
          prev_w=update_weight
          prev_b=update_biases
          
        t=t+1

        
        self.Weights=self.Weights-update_weight
        self.Biases=self.Biases-update_biases
        if anneal == True:
          Eta = ((0.99)**(start+3.5))*etap

        if self.Loss_Function=='CE':
          Loss_in_batch=Loss_in_batch+log_loss(Y_Train_curr_batch.T,Curr_Batch_Prediction)
        elif self.Loss_Function=='MSE':
          Loss_in_batch=Loss_in_batch+(mean_squared_error(Y_Train_curr_batch.T,Curr_Batch_Prediction))          
      

      Loss_in_Training.append(Loss_in_batch)
      Train_Prediction=self.Feed_Network(X_Train)
      print("Epoch:",Ep)
      print("Train Accuracy",self.Get_Accuracy(Y_Train.T,Train_Prediction))
      print("Loss in Training",Loss_in_Training[Ep])
      
      Validation_Predcition=self.Feed_Network(X_Valid)
      if self.Loss_Function=='CE':
        Loss_in_Validation.append(log_loss(Y_Valid.T,Validation_Predcition))
      elif self.Loss_Function=='MSE':
        Loss_in_Validation.append(mean_squared_error(Y_Valid.T,Validation_Predcition))
      print("Validation Accuracy",self.Get_Accuracy(Y_Valid.T,Validation_Predcition))
      print("Loss in Validation",Loss_in_Validation[Ep])
      wandb.log({"val_loss": Loss_in_Training[Ep]/10000, \
                       "val_acc": self.Get_Accuracy(Y_Train.T,Train_Prediction), \
                       "test_loss": Loss_in_Validation[Ep]/10000, \
                       "test_acc": self.Get_Accuracy(Y_Valid.T,Validation_Predcition), \
                       "epoch":Ep})





In [ ]:
def Main_Plots():
    sweep_config = {"name": "complete-sweep", "method": "grid"}
    sweep_config["metric"] = {"name": "accuracy", "goal": "minimize"}
    parameters_dict = {
                    "num_epochs": {"values": [5,10,20]}, \
                    "Layer_size": {"values": [32,64,128]}, \
                    "learning_rate": {"values": [1e-3,1e-4]}, \
                    "N_hl": {"values": [4]}, \
                    "optimizer": {"values": ["Momentum","GD","Nadam","Nag","RMSProp","Adam"]}, \
                    "batch_size": {"values": [32,64,128]}, \
                    "weight_init": {"values": ["Random","Xavier"]} , \
                    "activation": {"values": [ "Tanh","Sigmoid","Relu"]}, \
                    "loss": {"values": ["CE","MSE"]}, \
                      }
    sweep_config["parameters"] = parameters_dict
    global accuracy
    accuracy=0
    def train_nn(config = sweep_config):
        global accuracy
        with wandb.init(config = config):
            config = wandb.init().config
            wandb.run.name = "e_{}_loss_{}_opt_{}_bs_{}_init_{}_ac_{}".format(config.num_epochs,\
                                                                          config.loss,\
                                                                          config.optimizer,\
                                                                          config.batch_size,\
                                                                          config.weight_init,\
                                                                          config.activation,\
                                                                          )






            n_classes = 10
            seed =8790
            loss_function = config.loss
            Activation_gx = [config.activation, config.activation,config.activation,config.activation, 'SoftMax']
            neurons=config.Layer_size
            Layers = [784, neurons,neurons,neurons,neurons, n_classes]
            Network=Feed_Forward_NeuralNetwork(Layers,Activation_gx,loss_function,seed,Weight_init=config.weight_init) 
            Network.Main_Training_Algo(X_Valid,Y_Valid,X_Train,Y_Train,optimizer=config.optimizer,NEpochs=config.num_epochs,Batch_Size=config.batch_size,Momentum=0.9,Eta=config.learning_rate,anneal=True)
            Validation_Predcition=Network.Feed_Network(X_Valid)
            accuracy=Network.Get_Accuracy(Y_Valid.T,Validation_Predcition)

            wandb.log({"accuracy": accuracy})
    sweep_id = wandb.sweep(sweep_config, project = "Main_Plots")
    wandb.agent(sweep_id, function = train_nn)





In [ ]:
Main_Plots() #To create all the main plots

In [ ]:
def ConfusionMatrix():
  import matplotlib.pyplot as plt

  import plotly
  import seaborn as sns
  from sklearn.metrics import confusion_matrix
  sweep_config = {"name": "best-sweep", "method": "grid"}
  sweep_config["metric"] = {"name": "loss", "goal": "minimize"}
  parameters_dict = {
                    "num_epochs": {"values": [1]}, \
                    "Layer_size": {"values": [64]}, \
                    "learning_rate": {"values": [1e-3]}, \
                    "N_hl": {"values": [4]}, \
                    "optimizer": {"values": ["Nadam"]}, \
                    "batch_size": {"values": [64]}, \
                    "weight_init": {"values": ["Random"]} , \
                    "activation": {"values": [ "Tanh"]}, \
                    "loss": {"values": ["CE"]}, \
                      }
  sweep_config["parameters"] = parameters_dict
  global accuracy
  accuracy=0

  ####################################################################
  def train_nn(config = sweep_config):
      global accuracy
      with wandb.init(config = config):
          config = wandb.init().config
          wandb.run.name = "e_{}_loss_{}_opt_{}_bs_{}_init_{}_ac_{}".format(config.num_epochs,\
                                                                          config.loss,\
                                                                          config.optimizer,\
                                                                          config.batch_size,\
                                                                          config.weight_init,\
                                                                          config.activation,\
                                                                          )

        
        
        
        
          n_classes = 10
          seed =8790
          loss_function = config.loss
          Activation_gx = [config.activation, config.activation,config.activation,config.activation, 'SoftMax']
          neurons =config.Layer_size
          Layers = [784, neurons,neurons ,neurons,neurons, n_classes]
          Network=Feed_Forward_NeuralNetwork(Layers,Activation_gx,loss_function,seed,config.weight_init) 
          Network.Main_Training_Algo(X_Valid,Y_Valid,X_Train,Y_Train,config.optimizer,config.num_epochs,config.batch_size,Momentum=0.9,Eta=config.learning_rate,anneal=True)
          Y_test_pred=Network.Feed_Network(X_Test)


          E=np.argmax(Y_Test.T,axis=0)
          T=np.argmax(Y_test_pred,axis=0)
          cf_matrix = confusion_matrix(E,T)
          class_names=["Tshirt👕","Trouser👖","Pullover🧥","Dress👗","Coat🥼","Sandal👡","Shirt👔","Sneaker👟","Bag👜","AnkleBoot🥾"]
          wandb.log({"my_conf_mat_id" : wandb.plot.confusion_matrix( preds=T, y_true=E,class_names=class_names)})

  ####################################################################
  sweep_id = wandb.sweep(sweep_config, project = "Best_Plot")
  wandb.agent(sweep_id, function = train_nn)
  #################################################################### [markdown]

In [ ]:
ConfusionMatrix()#To create confusion matrix for best Hyperparameter config.

Create sweep with ID: l09zso2m
Sweep URL: https://wandb.ai/sahilb/Best_Plot/sweeps/l09zso2m


wandb: Agent Starting Run: 7wy06aol with config:
wandb: 	Layer_size: 64
wandb: 	N_hl: 4
wandb: 	activation: Tanh
wandb: 	batch_size: 64
wandb: 	learning_rate: 0.001
wandb: 	loss: CE
wandb: 	num_epochs: 1
wandb: 	optimizer: Nadam
wandb: 	weight_init: Random


Epoch: 0
Train Accuracy 62.831481481481475
Loss in Training 16346.043742106038
Validation Accuracy 62.61666666666667
Loss in Validation 4518.958496052425


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
